In [4]:
import requests, zipfile, io
url = "https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip"
response = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(response.content))
z.extractall("data")

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import os

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


### Data Preparation

The dataset contains around 1000 images of hairs in the separate folders 
for training and test sets. 

### Reproducibility

Reproducibility in deep learning is a multifaceted challenge that requires attention 
to both software and hardware details. In some cases, we can't guarantee exactly the same results during the same experiment runs.

Therefore, in this homework we suggest to set the random number seed generators by:

```python
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
```

Also, use PyTorch of version 2.8.0 (that's the one in Colab).

### Model

For this homework we will use Convolutional Neural Network (CNN). We'll use PyTorch.

You need to develop the model with following structure:

* The shape for input should be `(3, 200, 200)` (channels first format in PyTorch)
* Next, create a convolutional layer (`nn.Conv2d`):
    * Use 32 filters (output channels)
    * Kernel size should be `(3, 3)` (that's the size of the filter)
    * Use `'relu'` as activation 
* Reduce the size of the feature map with max pooling (`nn.MaxPool2d`)
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using `flatten` or `view`
* Next, add a `nn.Linear` layer with 64 neurons and `'relu'` activation
* Finally, create the `nn.Linear` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use `torch.optim.SGD` with the following parameters:

* `torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

In [37]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Question 1

Which loss function you will use?

* `nn.MSELoss()`
* `nn.BCEWithLogitsLoss()`
* `nn.CrossEntropyLoss()`
* `nn.CosineEmbeddingLoss()`

## Answer   `nn.BCEWithLogitsLoss()`  

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class HairCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(3, 32, kernel_size=3)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.conv(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [39]:
from torchinfo import summary

model = HairModel()  # stays on CPU
summary(model, input_size=(1, 3, 200, 200))



Layer (type:depth-idx)                   Output Shape              Param #
HairModel                                [1, 1]                    --
├─Conv2d: 1-1                            [1, 32, 198, 198]         896
├─MaxPool2d: 1-2                         [1, 32, 99, 99]           --
├─Linear: 1-3                            [1, 64]                   20,072,512
├─Linear: 1-4                            [1, 1]                    65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
Total mult-adds (M): 55.20
Input size (MB): 0.48
Forward/backward pass size (MB): 10.04
Params size (MB): 80.29
Estimated Total Size (MB): 90.81

### Question 2

What's the total number of parameters of the model? You can use `torchsummary` or count manually. 

In PyTorch, you can find the total number of parameters using:

```python
# Option 1: Using torchsummary (install with: pip install torchsummary)
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

# Option 2: Manual counting
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")
```

* 896 
* 11214912
* 15896912
* 20073473

### Question 3

What is the median of training accuracy for all the epochs for this model?

* 0.05
* 0.12
* 0.40
* 0.84 

In [40]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [41]:
data_dir = "C:/Users/HP/Desktop/Machine_Learning_Zoomcamp/machine-learning-zoomcamp-master/08-deep-learning/data/data"

train_dataset = datasets.ImageFolder(root=os.path.join(data_dir, "train"), transform=train_transforms)
test_dataset = datasets.ImageFolder(root=os.path.join(data_dir, "test"), transform=test_transforms)


In [42]:
from torch.utils.data import random_split

# Define sizes for train and validation
val_size = int(0.2 * len(train_dataset))  # 20% for validation
train_size = len(train_dataset) - val_size

# Split the dataset
train_dataset, validation_dataset = random_split(train_dataset, [train_size, val_size])

# Now create the data loaders
batch_size = 20

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



In [43]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
import torch.nn as nn

# Binary classification loss
criterion = nn.BCEWithLogitsLoss()



In [44]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")


Epoch 1/10, Loss: 0.6472, Acc: 0.6516, Val Loss: 0.6059, Val Acc: 0.6625
Epoch 2/10, Loss: 0.5449, Acc: 0.7063, Val Loss: 0.6885, Val Acc: 0.5813
Epoch 3/10, Loss: 0.4883, Acc: 0.7500, Val Loss: 0.5729, Val Acc: 0.7063
Epoch 4/10, Loss: 0.4191, Acc: 0.7953, Val Loss: 0.6047, Val Acc: 0.6813
Epoch 5/10, Loss: 0.3951, Acc: 0.8094, Val Loss: 0.6391, Val Acc: 0.6500
Epoch 6/10, Loss: 0.2811, Acc: 0.8906, Val Loss: 0.7502, Val Acc: 0.6687
Epoch 7/10, Loss: 0.2599, Acc: 0.8984, Val Loss: 0.6435, Val Acc: 0.6813
Epoch 8/10, Loss: 0.2022, Acc: 0.9141, Val Loss: 0.6747, Val Acc: 0.7312
Epoch 9/10, Loss: 0.1367, Acc: 0.9594, Val Loss: 0.6746, Val Acc: 0.7438
Epoch 10/10, Loss: 0.0859, Acc: 0.9844, Val Loss: 0.8910, Val Acc: 0.6875


In [45]:
train_accuracies = history['acc']
median_acc = np.median(train_accuracies)
print(f"Median training accuracy: {median_acc:.4f}")

Median training accuracy: 0.8500



### Question 4

What is the standard deviation of training loss for all the epochs for this model?

* 0.007
* 0.078
* 0.171
* 1.710

In [50]:
train_losses = [0.7181, 0.5645, 0.5177, 0.4679, 0.4112, 0.3610, 0.3578, 0.2131, 0.2322, 0.1697]
std_loss = np.std(train_losses)
print(f"Standard deviation of training loss: {std_loss:.3f}")

Standard deviation of training loss: 0.163


### Question 5 

Let's train our model for 10 more epochs using the same code as previously.

> **Note:** make sure you don't re-create the model.
> we want to continue training the model we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

* 0.008
* 0.08
* 0.88
* 8.88

In [55]:
from torchvision import transforms

train_transforms_aug = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Keep test transforms same as before
test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [56]:
from torchvision import datasets
from torch.utils.data import DataLoader
import os

data_dir = "C:/Users/HP/Desktop/Machine_Learning_Zoomcamp/machine-learning-zoomcamp-master/08-deep-learning/data/data"

train_dataset_aug = datasets.ImageFolder(root=os.path.join(data_dir, "train"), transform=train_transforms_aug)
test_dataset = datasets.ImageFolder(root=os.path.join(data_dir, "test"), transform=test_transforms)

batch_size = 20
train_loader_aug = DataLoader(train_dataset_aug, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [57]:
num_epochs_aug = 10  # 10 more epochs

test_losses_aug = []
test_accuracies_aug = []

for epoch in range(num_epochs_aug):
    model.train()
    for images, labels in train_loader_aug:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluate on test set
    model.eval()
    running_test_loss = 0.0
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_test_loss += loss.item() * images.size(0)

            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    epoch_test_loss = running_test_loss / len(test_dataset)
    epoch_test_acc = correct_test / total_test

    test_losses_aug.append(epoch_test_loss)
    test_accuracies_aug.append(epoch_test_acc)

    print(f"Epoch {epoch+1}/{num_epochs_aug} | Test Loss: {epoch_test_loss:.4f} | Test Acc: {epoch_test_acc:.4f}")

# Calculate mean test loss for Question 5
mean_test_loss = np.mean(test_losses_aug)
print(f"Mean test loss with augmentation: {mean_test_loss:.3f}")



Epoch 1/10 | Test Loss: 0.8082 | Test Acc: 0.5622
Epoch 2/10 | Test Loss: 0.6907 | Test Acc: 0.6269
Epoch 3/10 | Test Loss: 0.6555 | Test Acc: 0.6667
Epoch 4/10 | Test Loss: 0.5850 | Test Acc: 0.6617
Epoch 5/10 | Test Loss: 0.6139 | Test Acc: 0.6667
Epoch 6/10 | Test Loss: 0.6138 | Test Acc: 0.6567
Epoch 7/10 | Test Loss: 0.6212 | Test Acc: 0.6866
Epoch 8/10 | Test Loss: 0.6033 | Test Acc: 0.6965
Epoch 9/10 | Test Loss: 0.5619 | Test Acc: 0.6866
Epoch 10/10 | Test Loss: 0.5741 | Test Acc: 0.7015
Mean test loss with augmentation: 0.633


### Question 6

What's the average of test accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?

* 0.08
* 0.28
* 0.68
* 0.98

In [58]:
mean_test_acc_last5 = np.mean(test_accuracies_aug[-5:])
print(f"Average test accuracy last 5 epochs: {mean_test_acc_last5:.3f}")


Average test accuracy last 5 epochs: 0.686
